In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import warnings
import sys
import gc
import torch
warnings.filterwarnings("ignore")
base_dir = os.path.split(os.getcwd())[0]
sys.path.append(base_dir)

from pyfunctions.local_importance import *
from pyfunctions._integrated_gradients import configure_interpretable_embeddings, remove_interpretable_embeddings

In [3]:
# set up env
#!python3.12 -m pip install lime
#!python3.12 -m pip install shap
#!python3.12 -m pip install tf-keras==2.16.0
#!python3.12 -m pip install captum

In [4]:
torch.autograd.set_grad_enabled(False)

In [147]:
device = 'cuda:0'
supported_model_and_task = {"bert": ["pathology", "sst2", "agnews"], "gpt2": ["ioi"]}
model_type = 'bert' #"bert", "gpt2"
data_name = supported_model_and_task[model_type][2] #"bert""pathology", "sst2", "agnews", "ioi"
method = "CDT" #"CDT", "lime", "shap", "LIG"

# todo: figure out proper max len for each dataset
if data_name == "sst2":
    max_seq_len = 120
    class_names = ["0", "1"]
if data_name == "agnews":
    max_seq_len = 240
    class_names = ["0", "1", "2", "3"]
elif data_name == "pathology":
    max_seq_len = 512
    class_names = ["3", "4", "5"]
elif data_name == "ioi":
    max_seq_len = 16
    class_names = [str(token_id) for token_id in range(50257)]

In [148]:
# load in data and model
data, le_dict, tokenizer, model = load_data_and_model(data_name, model_type, device)

In [158]:
sample_id = 104

if data_name != "ioi":
    tokenized_prompt = None
    io_seq_idx = None
    s_seq_idx = None
    text = data['docs'][sample_id]
    label = data['labels'][sample_id]
    label_idx = le_dict[label]
    tokenizer.pad_token = tokenizer.eos_token
else:
    tokenized_prompt = data.tokenized_prompts[sample_id]
    io_seq_idx = data.io_tokenIDs[sample_id] # sample_id 0-50 for ioi
    s_seq_idx = data.s_tokenIDs[sample_id] 
    text = data.sentences[sample_id]
    label_idx = None

In [159]:
# clear cache
gc.collect()
torch.cuda.empty_cache()

In [160]:
# passed: LIG, shap, lime, cdt

scores = run_local_importance(text, tokenized_prompt, io_seq_idx, s_seq_idx, label_idx, max_seq_len, model, tokenizer,
                              device, method, model_type, class_names)

Running decomposition in batches...: 100%|██████████| 1/1 [00:00<00:00,  1.27it/s]

Viz rel: 


Viz irrel: 


Viz rel-irrel: 


In [67]:
cnt = 0
for x in data['docs']:
    cnt += len(x)
print(cnt/len(data['docs']))

235.2992105263158


In [72]:
le_dict[label]

3